In [1]:
exec(open('/choreonoid_ws/install/share/irsl_choreonoid/sample/irsl_import.py').read())
import yaml
from irsl_choreonoid_ros.setup_cnoid import SetupCnoid

## シーンのロード

In [2]:
info=yaml.safe_load(open('/choreonoid_ws/install/share/irsl_sim_environments/cnoid/world/lake_house_world.yaml'))
cnoid=SetupCnoid(); cnoid.buildEnvironment(info, createWorld=True, setCamera=True)

## ロボットの導入

In [3]:
fname = 'models/JVRC-1/main.wrl'
rb = ib.loadRobotItem(fname, world=False) ## in choreonoid GUI
robot = RobotModel(rb)

## 便利関数

In [119]:
def makeListInterpolation(start, end, size=10):
    res = [ start ]
    for i in range(size-1):
        rb_ = (i + 1)/size
        ra_ = (size - i -1)/size
        lst_=[]
        for a, b in zip(start, end):
            lst_.append(ra_ * a + rb_ * b)
        res.append(lst_)
    res.append(end)
    return res

def makeVectorInterpolation(start, end, size=10):
    res = [ start ]
    for i in range(size-1):
        rb_ = (i + 1)/size
        ra_ = (size - i -1)/size
        res.append(ra_ * start + rb_ * end)
    res.append(end)
    return res

def makeCoordsInterpolation(start, end, size=10):
    res = [ start.copy() ]
    for i in range(size-1):
        ratio_ = (i + 1)/size
        res.append(start.mid_coords(ratio_, end))
    res.append(end.copy())
    return res

### カメラ位置の設定

In [5]:
def setCameraPosition(x, y, theta, z=0.0, eye_height=1.7, view_height=0.7, view_x=1.2, view_y=0.0, fov=None, robot=None, robotOffset=[0,0,0.8]):
    eye_ = npa([x, y, z + eye_height])
    up_ = coordinates.Z
    stand_cds = coordinates(npa([x, y, z])).rotate(theta, coordinates.Z)
    if robot is not None:
        rcds = stand_cds.copy().translate(npa(robotOffset))
        robot.rootCoords(rcds)
    at_ = stand_cds.transform_vector(npa([view_x, view_y, view_height]))
    cam_ = ib.cameraPositionLookingAt(eye_, at_, up_)
    ib.setCameraCoords(cam_, fov=fov, opencv=False)

### ロボット位置の設定

In [6]:
def setRobotPosition(x, y, theta, z=0.0, robot=None, robotOffset=[0,0,0.8]):
    stand_cds = coordinates(npa([x, y, z])).rotate(theta, coordinates.Z)
    if robot is not None:
        rcds = stand_cds.copy().translate(npa(robotOffset))
        robot.rootCoords(rcds)

In [7]:
def setRobotPosition(x, y, theta, z=0.0, robot=None, robotOffset=[0,0,0.8]):
    stand_cds = coordinates(npa([x, y, z])).rotate(theta, coordinates.Z)
    if robot is not None:
        rcds = stand_cds.copy().translate(npa(robotOffset))
        robot.rootCoords(rcds)

### 位置の取得

In [70]:
def getPosition(coords):
    x = coords.pos[0]
    y = coords.pos[1]
    theta = coords.RPY[2]
    return [x, y, theta]

def getCameraPosition():
    cds, fov = ib.getCameraCoords()
    zz=cds.z_axis
    return [ cds.pos[0], cds.pos[1], math.atan2(zz[1], zz[0]) ]

def getRobotPosition(robot):
    cds = robot.rootCoords()
    return getPosition(cds)

### データのセーブロード

In [100]:
import pickle
def writeData(fname, robot):
    cds_, fov_ = ib.getCameraCoords()
    data_ = (getCameraPosition(), getRobotPosition(robot=robot), robot.angleVector(), 
             ru.make_coords_map(cds_), ru.make_coords_map(robot.rootCoords()), fov_)
    with open(fname, mode='wb') as f:
        pickle.dump(data_, f)

def readData(fname):
    res_ = None
    with open(fname, mode='rb') as f:
        res_ = pickle.load(f)
    return res_

### データを使う

In [128]:
def applyData(data_):
    ib.setCameraCoords(ru.make_coordinates(data_[3]))
    robot.angleVector(data_[2])
    robot.rootCoords(ru.make_coordinates(data_[4]))

In [146]:
def applyOldData(data_):
    if len(data_) < 5:
        setCameraPosition(*(data_[0]))
        setRobotPosition(*(data_[1]), robot=robot)
        robot.angleVector(data_[2])

In [123]:
#[('file0', 0.1)
# ('file1', 0.1)
# ('fileN', 0.1)
# ]

def makeImages(lst, robot=None, rate=10):
    if robot is None:
        robot = eval('robot')
    cntr = 0
    prev_data_ = readData(lst[0][0])
    for fname, tm in lst[1:]:
        data_ = readData(fname)
        size = int(tm * rate)
        # d0 = makeListInterpolation(prev_data_[0], data_[0], size=size)
        # d1 = makeListInterpolation(prev_data_[1], data_[1], size=size)
        d0 = makeCoordsInterpolation(ru.make_coordinates(prev_data_[3]),
                                     ru.make_coordinates(data_[3]), size=size)
        d1 = makeCoordsInterpolation(ru.make_coordinates(prev_data_[4]),
                                     ru.make_coordinates(data_[4]), size=size)
        d2 = makeVectorInterpolation(prev_data_[2], data_[2], size=size)
        for l0, l1, v0 in zip(d0, d1, d2):
            #setCameraPosition(l0*)
            #setRobotPosition(l0*, robot=robot)
            print(l0)
            print(l1)
            print(v0)
            ib.setCameraCoords(l0)
            robot.rootCoords(l1)
            robot.angleVector(v0)
            fname='image{:0=6}.png'.format(cntr)
            cntr += 1
            ib.saveImageOfScene(fname)
        prev_data_ = data_

### 関数の実行

カメラ位置

In [9]:
getCameraPosition()

[4.625419607741056, -0.3973106629012421, 1.960796318077717]

ロボット位置

In [10]:
getRobotPosition(robot)

[0.0, 0.0, 0.0]

セーブ

In [141]:
ib.saveImageOfScene('scene000.png')

ロボット位置

In [113]:
setRobotPosition(0, 0, PI/2, robot=robot)

カメラ位置

In [126]:
setCameraPosition(2, 0, PI, )

ポーズの補間

In [27]:
lst=makeRangeList([0,0,0],[0,0,PI], size=20)
for idx, l in enumerate(lst):
    setRobotPosition(*l, robot=robot)
    ib.saveImageOfScene('scene{}.png'.format(idx))

データの保存

In [101]:
writeData('data000.pyc', robot=robot)

In [102]:
readData('data000.pyc')

([2.0, 0.0, 3.141592653589793], [0.0, 0.0, 1.5707963267948963], array([0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.06212983,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        ]), {'pos': [2.0, 0.0, 1.7], 'aa': [-0.6712505373490204, -0.6712505373490202, 0.3143969341728753, 2.5323684502952744]}, {'pos': [0.0, 0.0, 0.8], 'aa': [0.0, 0.0, 1.0, 1.5707963267948966]}, 0.75)

画像サイズのセット

In [140]:
ib.currentSceneWidget().setScreenSize(800,600)